<a href="https://colab.research.google.com/github/sayalikamble4567/2026-BUS4-118S-Sec-02-Special-Topics-MIS/blob/main/Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
# Generate sample customer data generated using chatgpt
df = pd.read_excel("/content/drive/MyDrive/Customer_Features_Dataset_Realistic.xlsx")

# Preprocess data: Select numerical features and scale them
features = ['annual_spending', 'purchase_frequency', 'age']
X = df[features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Determine optimal number of clusters using elbow method
inertia = []
K = range(1, 6)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)
# Plot elbow curve
plt.figure(figsize=(8, 5))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal K')
plt.savefig('elbow_plot.png')
plt.close()
# Apply K-Means with optimal K (e.g., 3 based on elbow method graph)
optimal_k = 3
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df['cluster'] = kmeans.fit_predict(X_scaled)
# added cluster sizes
print("\nCluster Sizes:")
print(df['cluster'].value_counts().sort_index())
#  added Silhouette Score for better validation
sil_score = silhouette_score(X_scaled, df['cluster'])
print("\nSilhouette Score:", round(sil_score, 3))
# Analyze clusters
cluster_summary = df.groupby('cluster')[features].mean().round(2)
print("Cluster Characteristics:")
print(cluster_summary)
# Example of targeted strategies
for cluster in range(optimal_k):
    print(f"\nCluster {cluster} Strategy:")
    if cluster_summary.loc[cluster, 'purchase_frequency'] > 20:
        print("High Spending Customer: Use subscription plans and Loyalty Program.")
    elif cluster_summary.loc[cluster, 'purchase_frequency'] > 10:
        print("Moderately Frequent Buyer but Low Spending: Provide bulk discounts and Use Discounts like BOGOs or Buy 1 Get 1 Half Off.")
    else:
        print("Low Frequency Buyer but Moderate Spending: Offer exclusive promotions and Send personalized re-engagement campaigns.")
# Save cluster assignments to CSV
df.to_csv('customer_segments.csv', index=False)

# Based on the K-Means clustering using annual spending, purchase frequency,
# and age customer segmentation the clusters separate customers into
# high-value frequent buyers (high spending and high frequency) and moderate
# customers (mid-level spending and frequency) (there is no low-engagement group
# because there are 2 moderately performing groups).
# High-value customers should be targeted with loyalty programs, VIP perks,
# referrals, and personalized upselling to maximize lifetime value while
# moderate customers represent growth potential and can be encouraged with
# bundles, limited-time offers, and targeted promotions to increase frequency
# and basket size. This can be seen in my reccomendations.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Cluster Sizes:
cluster
0    436
1    196
2    368
Name: count, dtype: int64

Silhouette Score: 0.374
Cluster Characteristics:
         annual_spending  purchase_frequency    age
cluster                                            
0                 856.72               10.18  28.80
1                2401.38               20.38  41.11
2                 957.17                8.76  54.71

Cluster 0 Strategy:
Moderately Frequent Buyer but Low Spending: Provide bulk discounts and Use Discounts like BOGOs or Buy 1 Get 1 Half Off.

Cluster 1 Strategy:
High Spending Customer: Use subscription plans and Loyalty Program.

Cluster 2 Strategy:
Low Frequency Buyer but Moderate Spending: Offer exclusive promotions and Send personalized re-engagement campaigns.
